<a href="https://colab.research.google.com/github/JGMFC/DL_KC/blob/main/Pr%C3%A1ctica_DL_JG_v2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Importación Librerías

In [252]:
import cv2
import numpy as np
import numpy
import pandas
import pandas as pd
import imageio.v3 as io
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout, concatenate
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import VGG16, imagenet_utils
from tensorflow.keras.regularizers import l2
import cv2
import imageio as io
from google.colab import drive
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from tqdm import tqdm
from typing import Optional, Union

## 1. Descarga y preprocesado de los datos

### 1.1 Descarga y formatting del CSV de los datos

In [253]:
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

--2024-03-03 13:01:13--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.248.20.69, 34.249.199.226
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.248.20.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [         <=>        ]  52.85M  12.2MB/s    in 5.2s    

2024-03-03 13:01:19 (10.3 MB/s) - ‘airbnb-listings.csv’ saved [55414009]



In [254]:
from tqdm import tqdm
from typing import Optional, Union

In [255]:
data = pandas.read_csv("airbnb-listings.csv", sep = ';')
data = data.dropna(subset=['Price'])
data

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,17823671,https://www.airbnb.com/rooms/17823671,20170407214119,2017-04-08,Room in beautiful flat in Madrid centre!! SC8_1_2,Great room with large bed and two windows,Amazing 12 bedroom flat with 3 complete bathro...,Great room with large bed and two windows Amaz...,none,"The "" Barrio de las Letras "" is one of the mos...",...,NaN,NaN,NaN,NaN,NaN,strict,20,NaN,"40.41487093316894,-3.6965780975998435","Host Has Profile Pic,Host Identity Verified,Re..."
1,17309526,https://www.airbnb.com/rooms/17309526,20170407214050,2017-04-08,Double room in Gracia/Whole Apartment,Double room in Gracia - Habitación doble en Gr...,NaN,Double room in Gracia - Habitación doble en Gr...,none,NaN,...,NaN,NaN,NaN,NaN,NaN,flexible,1,NaN,"41.40132396962952,2.163712769014997","Host Has Profile Pic,Requires License"
2,801113,https://www.airbnb.com/rooms/801113,20170407214119,2017-04-08,DOUBLE ROOM WITH PRIVATE BATHROOM IN LA LATINA,Bright and spacious double room and bathroom f...,Spacious double room with private bathroom and...,Bright and spacious double room and bathroom f...,none,".Safety neighborhood with all the amenities, g...",...,9.0,10.0,9.0,NaN,NaN,moderate,1,3.15,"40.40860126037668,-3.7145957909729637","Host Has Profile Pic,Is Location Exact,Require..."
3,7357466,https://www.airbnb.com/rooms/7357466,20170407214119,2017-04-08,Alquiler habitación mes de agosto,"Hola, somos una familia de 5 amigos que vivímo...","La habitación que alquilamos es amplia, lumino...","Hola, somos una familia de 5 amigos que vivímo...",none,El piso es muy amplio y las zonas comunes dan ...,...,NaN,NaN,NaN,NaN,NaN,flexible,1,NaN,"40.43408638806261,-3.654979849617808","Host Has Profile Pic,Host Identity Verified,Re..."
4,17050866,https://www.airbnb.com/rooms/17050866,20170407214119,2017-04-08,Habitacion a solo 3 min de atocha,Nuestro Habitacion esta hubicada en centro de...,NaN,Nuestro Habitacion esta hubicada en centro de...,none,Es un barrio céntrico de madrid . Donde hay mu...,...,10.0,10.0,10.0,NaN,NaN,flexible,1,0.71,"40.44181649452356,-3.6451428471469227","Host Has Profile Pic,Requires License"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13996,16231125,https://www.airbnb.com/rooms/16231125,20170407214119,2017-04-08,"Piso en en centro de Madrid,Quevedo","Piso de 3 habitaciones, un salón, un baño y un...",NaN,"Piso de 3 habitaciones, un salón, un baño y un...",none,Está muy bien comunicado y muy céntrico. Ademá...,...,NaN,NaN,NaN,NaN,NaN,moderate,1,NaN,"40.4348724447834,-3.707547039320753","Host Has Profile Pic,Requires License"
13997,4741021,https://www.airbnb.com/rooms/4741021,20170407214119,2017-04-08,"OLAVIDE IV, 3ºD, Friendly Rentals",The apartment is in a characteristic building ...,This lovely apartment is one of several we can...,The apartment is in a characteristic building ...,none,This very safe residential district is surroun...,...,8.0,NaN,NaN,NaN,NaN,strict,145,0.17,"40.43613375660163,-3.7027578557736502","Host Has Profile Pic,Is Location Exact,Require..."
13998,5803594,https://www.airbnb.com/rooms/5803594,20170407214119,2017-04-08,Habitación pequeña cómoda amena tranquila.,cocina para utilizar para comidas muy básicas ...,"HABITACIÓN privada pequeña pero cómoda,apartam...",cocina para utilizar para comidas muy básicas ...,none,Barrio de Madrid Ubicado cerca plaza de toros...,...,9.0,8.0,9.0,NaN,NaN,strict,1,2.10,"40.423445712695965,-3.645238780837596","Host Has Profile Pic,Requires License"
13999,16025541,https://www.airbnb.com/rooms/16025541,20170407214119,2017-04-08,New reformed apartment. Everything new!!!!,Outside vieuw without noise. Ideal for two per...,NaN,Outside vieuw without noise. Ideal for two per.

In [256]:
# ¿De qué variables disponemos?
data.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

### 1.2. Descarga de imágenes

In [257]:
def download_images(paths: list,
                    canvas: tuple = (224, 224),
                    nb_channels: int = 3,
                    max_imgs: Optional[int] = None
                    ) -> tuple:
  """ Download a list of images from url adresses, converting them to a specific
  canvas size.

  Args:
    paths: Paths or url adresses from which to load images.
    canvas: Desired image width and height.
    nb_channels: Channels in images (1 for B/W, 3 for RGB).
    max_imgs: Upper threshold in the number of images to download.

  Return:
    a tuple of:
      - image values
      - indices within the paths that were successfull.

  """
  n_images = len(paths) if not max_imgs else max_imgs
  images = numpy.zeros((n_images, canvas[0], canvas[1], nb_channels),
                       dtype=numpy.uint8)
  downloaded_idxs = []

  for i_img, url in enumerate(tqdm(paths, total=n_images)):
    if i_img >= n_images:
      break
    try:
        img = io.imread(url)
        img = cv2.resize(img, (canvas[0], canvas[1]))
        downloaded_idxs.append(i_img)
        images[i_img] = img
    except (IOError, ValueError) as e:  # Unavailable url / conversion error
        pass
  return images[downloaded_idxs], downloaded_idxs

Normalización y conversión en el rango [0, 1]

In [ ]:
images, idxs = download_images(data['Thumbnail Url'], max_imgs=3000)
images = images.astype("float32") / 255.
images.shape

  0%|          | 0/3000 [00:00<?, ?it/s]<ipython-input-257-4b4849c24d37>:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = io.imread(url)
  8%|▊         | 230/3000 [02:45<36:21,  1.27it/s]

Filtrado acorde en los demás datos

In [ ]:
filtered_data = data.iloc[idxs]
filtered_data

### 1.3 Guardado de los datos para posterior uso/selección

In [ ]:
numpy.save('images.npy', images)
filtered_data.to_csv('filtered_data.csv', sep=';', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp images.npy /content/drive/MyDrive/images_final.npy
!cp filtered_data.csv /content/drive/MyDrive/filtered_data.csv

!ls -lah images* filtered*  # Comprobación

In [ ]:
# Comprobamos a abrirlos de nuevo
saved_data = pandas.read_csv("/content/drive/MyDrive/filtered_data.csv", sep=';')
saved_imgs = numpy.load("/content/drive/MyDrive/images_final.npy")
saved_data.shape, saved_imgs.shape

Así, por ejemplo, cargamos y visualizamos la distribución de las etiquetas:

In [ ]:
data['Price'].hist(bins=100)

Y podemos dividir por porcentajes, para crear clases, como por ejemplo:

In [ ]:
data['Price'].describe()

In [ ]:
y_class = []
for y in data['Price']:
  if y <= 32:
    y_class.append(0) # Rango más barato
  elif 32 < y <= 55:
    y_class.append(1) # Rango intermedio
  else:
    y_class.append(2)
y_class = pandas.Series(y_class)
y_class.hist(bins=3)

Al dividir entre train y test, mantener la coherencia con las imagenes
Funcion de perdidas no puede tener not a number

## 3. División entre train y test

Dividimos los datos entre train test y validación. en proporción 80-20 y a su vez 50-50

In [ ]:
from sklearn.model_selection import train_test_split

train, test   = train_test_split(saved_data, test_size=0.2, shuffle=True, random_state=1234)
test, valid   = train_test_split(test, test_size=0.5, shuffle=True, random_state=1234)

print(f'Shape 1D train: {train.shape}')
print(f'Shape 1D valid: {valid.shape}')
print(f'Shape 1D test:  {test.shape}')

Declaramos los indices para asegurar la coherencia entre train-valid-test de 1D e imagenes

In [ ]:
train_index , valid_index , test_index = train.index , valid.index , test.index

Para facilitar su recuperación y dado el tamaño guardardamos por separado y nos aseguramos que mantengan las dimensiones anteriores

In [ ]:
saved_data.loc[train_index].to_csv('/content/drive/MyDrive/train_1D.csv', sep=';', decimal='.', index=False)
saved_data.loc[valid_index].to_csv('/content/drive/MyDrive/valid_1D.csv', sep=';', decimal='.', index=False)
saved_data.loc[test_index].to_csv('/content/drive/MyDrive/test_1D.csv', sep=';', decimal='.', index=False)

print(f'Shape 1D train CSV: {saved_data.loc[train_index].shape}')
print(f'Shape 1D valid CSV: {saved_data.loc[valid_index].shape}')
print(f'Shape 1D test CSV:  {saved_data.loc[test_index].shape}')

Separamos en base a los indices anteriores las imagenes

In [ ]:
np.save('/content/drive/MyDrive/train_images.npy',saved_imgs[train_index])
np.save('/content/drive/MyDrive/valid_images.npy',saved_imgs[valid_index])
np.save('/content/drive/MyDrive/test_images.npy',saved_imgs[test_index])


print(f'Shape images train CSV: {saved_imgs[train_index].shape}')
print(f'Shape images valid CSV: {saved_imgs[valid_index].shape}')
print(f'Shape images test CSV:  {saved_imgs[test_index].shape}')

## 4. Análisis de los datos (Partes extraídas de la práctica de ML)

In [ ]:
# Montamos (al ejecutarse en distintos dias el Drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Cargamos el dataset de train

df = pd.read_csv('/content/drive/MyDrive/train_1D.csv', sep=';', decimal='.')

In [ ]:
# Hacemos un drop de todas las columnas que OBVIAMENTE no aportan información relevante
not_useful_columns = [
    'ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Thumbnail Url', 'Medium Url',
    'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name',
    'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood', 'Weekly Price',
    'Monthly Price', 'Calendar Updated', 'Calendar last Scraped', 'First Review',
    'Last Review', 'Reviews per Month', 'Geolocation', 'Calculated host listings count',
    'Host Listings Count', 'Host Total Listings Count']

df.drop(not_useful_columns, axis=1, inplace=True)

Hay una serie de columnas que, si bien aportan mucha información, exigen un tratamiento previo que queda fuera del alcance de este trabajo. Name, summary, description... exigirían tokenización y un análisis con NLP para poder sacar algo en claro, de modo que se eliminan.

In [ ]:
# Hacemos un drop de todas las columnas que aportando información quedan fuera del scope
NLP_columns = ['Name', 'Summary', 'Space', 'Description',
       'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Host Location', 'Host About']

df.drop(NLP_columns, axis=1, inplace=True)

Analizamos los valores

In [ ]:
df.apply(lambda x: len(x.unique()))

La variable Street no muestra realmente la calle; en muchas, simplemente es la ciudad y un código postal (variables que ya tienen su propia columna). Es una variable muy ruidosa y no categorizada.

Por otro lado, State, Market y Smart Location aportan la misma información, al igual que Country y Country Code y de forma similar a Neighbourhood y Zipcode.

In [ ]:
df.drop(['Street', 'State', 'Market', 'Smart Location', 'Country', 'Zipcode'], axis=1, inplace=True)

In [ ]:
unique_values = df['City'].value_counts()
print(unique_values)

Vemos que mayoritariamente está Madrid, por lo que vamos a filtrar por esta columna y luego eliminarla. Esto Restringe nuestro análisis/Modelo a la ciudad de Madrid

In [ ]:
    df = df[df['City'] == 'Madrid']
    df.drop(['City'], axis=1, inplace=True)

In [ ]:
df.head()

Por último se comprueba qué variables tienen muchos valores missing (NaN or null). Hay columnas en las que apenas hay datos

In [ ]:
print(f'El dataframe tiene un total de {len(df)} filas.')
len(df) - df.count() # vemos cuántos valores NaN hay por columna

In [ ]:
NaN_values = [
    'Host Acceptance Rate', 'Square Feet', 'Has Availability', 'License', 'Jurisdiction Names'
]
df.drop(NaN_values, axis=1, inplace=True)

Analizamos su hay variables redundantes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Compute the correlation matrix
corr = np.abs(df.drop(['Price'], axis=1).corr())

# Genera una máscara para la parte inferior de la matriz
mask = np.triu(corr)

# Configura la figura de matplotlib
f, ax = plt.subplots(figsize=(12, 10))

# Dibuja el mapa de calor con la máscara y la relación de aspecto correcta
sns.heatmap(corr, mask=mask, vmin=0.0, vmax=1.0, center=0.5,
            linewidths=0.1, cmap="YlGnBu", cbar_kws={"shrink": 0.8})

plt.show()

# puedo representar solo la parte inferior ya que es simétrica

Vemos que hay mucha correlación entre las variables de availability y las de reviews. Podemos eliminar todas las availability salvo 365 y hacer una media de las de review; he decidido hacer media porque son reviews. Eliminamos también Accommodates que tiene correlación alta con Beds.

In [ ]:
reviews = df.loc[: , "Review Scores Rating":"Review Scores Value"]
df['Average Review'] = reviews.mean(axis=1)

df.drop(['Availability 30', 'Availability 60', 'Availability 90', 'Review Scores Rating',
         'Review Scores Accuracy', 'Review Scores Cleanliness', 'Review Scores Checkin',
       'Review Scores Communication', 'Review Scores Location', 'Review Scores Value',
        'Accommodates'], axis=1, inplace=True)

In [ ]:
# Compute the correlation matrix
corr = np.abs(df.drop(['Price'], axis=1).corr())

# Genera una máscara para la parte inferior de la matriz
mask = np.triu(corr)

# Configura la figura de matplotlib
f, ax = plt.subplots(figsize=(12, 10))

# Dibuja el mapa de calor con la máscara y la relación de aspecto correcta
sns.heatmap(corr, mask=mask, vmin=0.0, vmax=1.0, center=0.5,
            linewidths=0.1, cmap="YlGnBu", cbar_kws={"shrink": 0.8})

plt.show()

In [ ]:
df.describe().T

In [ ]:
df_n = df[['Neighbourhood', 'Neighbourhood Cleansed', 'Neighbourhood Group Cleansed']]
df_n.head(20)

In [ ]:
print(f'El dataframe tiene un total de {len(df_n)} filas.')
len(df_n) - df_n.count() # vemos cuántos valores NaN hay por columna

In [ ]:
df_n.apply(lambda x: len(x.unique())) # vemos el número de valores distintos por columna

Debido a que Neighbourhood Cleansed no tiene nulos pero tiene muchos valores distintos se va a usar Neighbourhood Group Cleansed a no ser que sea NaN/Null; en ese caso se rellenará con Neighbourhood Cleansed

In [ ]:
df_n["Final Neighbourhood"] = df_n["Neighbourhood Group Cleansed"].fillna(df_n["Neighbourhood Cleansed"])

In [ ]:
df["Final Neighbourhood"] = df["Neighbourhood Group Cleansed"].fillna(df["Neighbourhood Cleansed"])
df.drop(["Neighbourhood", "Neighbourhood Group Cleansed", "Neighbourhood Cleansed"], axis=1, inplace=True)

Host Verifications, Features y Amenities exigirían un análisis NLP, pero hay información directa que se puede extraer. Todas son listas, separadas por comas. Se puede contar el número de elementos en cada lista y usarlo como característica.

In [ ]:
df_l = df[['Features', 'Amenities', 'Host Verifications']]
df_l.fillna("", inplace=True) # rellenamos los huecos con cadenas vacías

df_l['Features'] = df_l['Features'].apply(lambda x: len(str(x).split(',')))
df_l['Amenities'] = df_l['Amenities'].apply(lambda x: len(str(x).split(',')))
df_l['Host Verifications'] = df_l['Host Verifications'].apply(lambda x: len(str(x).split(',')))

df_l.head(10)

In [ ]:
df['Features'].fillna("", inplace=True)
df['Amenities'].fillna("", inplace=True)
df['Host Verifications'].fillna("", inplace=True)

df['Features'] = df['Features'].apply(lambda x: len(str(x).split(',')))
df['Amenities'] = df['Amenities'].apply(lambda x: len(str(x).split(',')))
df['Host Verifications'] = df['Host Verifications'].apply(lambda x: len(str(x).split(',')))

In [ ]:
df.columns

In [ ]:
from datetime import datetime

df.dropna(subset=['Host Since'], inplace=True) # se ha visto antes que solo hay tres valores missing, no se pierde mucho

df['Host Since'] = df['Host Since'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))
df['Years Being Host'] = df['Host Since'].apply(lambda x: 2017 - x.year)

df.drop(['Host Since'], axis=1, inplace=True)

Categorización de Variables: Vemos el tipo de las columnas restantes. Las numéricas se convierten a float, y se va a hacer una limpieza general: cero baños, precio cero... esas cosas pueden y deben limpiarse al recoger los datos.

In [ ]:
print(f"Registros con cero baños: {len(df[df['Bathrooms'] == 0])}")
print(f"Registros con precio cero: {len(df[df['Price'] == 0])}")

df = df[df['Bathrooms'] != 0]
df = df[df['Price'] != 0]

In [ ]:
list(df.select_dtypes(include=['object']).columns)

In [ ]:
df_nn = df[list(df.select_dtypes(include=['object']).columns)]
df_nn.apply(lambda x: len(x.unique())) # valores diferentes

In [ ]:
len(df_nn) - df_nn.count() # solo market y host response time tienen missing values

In [ ]:
df_nn["Host Response Time"].value_counts()

In [ ]:
df["Host Response Time"].fillna(df["Host Response Time"].mode()[0], inplace=True)

In [ ]:
df["Experiences Offered"].value_counts()

In [ ]:
df.drop(['Experiences Offered'], axis=1, inplace=True)

In [ ]:
df.drop(['Country Code'], axis=1, inplace=True)

In [ ]:
categorical = ['Host Response Time', 'Property Type', 'Room Type', 'Bed Type', 'Cancellation Policy',
 'Final Neighbourhood'
]

# Creamos un dict para usarlo después en test
mean_map = {}
for c in categorical:
    mean = df.groupby(c)['Price'].mean()
    df[c] = df[c].map(mean)
    mean_map[c] = mean

In [ ]:
cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Price')))
df = df[cols]

df.fillna(df.mean(), inplace=True)
df.isnull().any()

La longitud y longitud nos indican la geolocalización de los apartamentos. Como en el procesado nos hemos quedado con las ubicaciones de madrid, calcularemos la distancia que hay desde el centro de Madrid hasta el apartamento.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
	""" Calculo distancia haversine entre dos localizaciones

    Parámetros:
	lat1 -- latitud posición 1
	lon1 -- longitud posicion 1
	lat2 -- latitud posición 2
	lon2 -- longitud posición 2

    Returns:
	Distancia (en m) entre dos puntos
	"""
	lon1 = np.radians(lon1)
	lat1 = np.radians(lat1)
	lon2 = np.radians(lon2)
	lat2 = np.radians(lat2)

	r = 6371

	dlon = np.subtract(lon2, lon1)
	dlat = np.subtract(lat2, lat1)

	a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),
               np.multiply(np.cos(lat1),
                           np.multiply(np.cos(lat2),
                                       np.power(np.sin(np.divide(dlon, 2)), 2))
                           )
              )
	c = np.multiply(2, np.arcsin(np.sqrt(a)))

	return c*r*1000

## 5. Preprocesado de los datos (en base al anterior apartado)

In [ ]:
def elimina_columnas(df,columnas):
	""" Elimina columna(s) de dataframe

	Parámetros:
	df -- dataframe a procesar
    columnas -- columna(s) a eliminar

	Returns:
	df - dataframe procesado por lo criterios optados en el procesamiento
	"""
	df = df.drop(columnas, axis=1)
	return df

def prepoc_df(df,imagenes):
	""" Procesamiento del dataframe con la ingeniería de características hecha en el procesamiento

	Parámetros:
	df - dataframe a procesar

	Returns:
	df - dataframe procesado por lo criterios optados en el procesamiento
	images - imagenes asociadas al dataframe
	"""


	#Longitud y Latitud centro madrid para calcular la distancia haversine
	centro_madrid_lat = 40.41831
	centro_madrid_lon = -3.70275


	not_useful_columns = [
			'ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Thumbnail Url', 'Medium Url',
			'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name',
			'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood', 'Weekly Price',
			'Monthly Price', 'Calendar Updated', 'Calendar last Scraped', 'First Review',
			'Last Review', 'Reviews per Month', 'Geolocation', 'Calculated host listings count',
			'Host Listings Count', 'Host Total Listings Count'
	]

	somehow_useful_columns_but_droppable_anyway = ['Name', 'Summary', 'Space', 'Description',
				'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
				'Host Location', 'Host About']


	not_useful_columns_2 = ['Street', 'State', 'Market', 'Smart Location', 'Country', 'Zipcode']

	columns_with_too_many_missing_values = [
			'Host Acceptance Rate', 'Square Feet', 'Has Availability', 'License', 'Jurisdiction Names'
	]

  #Quitamos las columnas con su mayoría de variables a nulos
	df = elimina_columnas(df,not_useful_columns)
  #Quitamos aquellas variables que no aprotan valor a la solución del problema
	df = elimina_columnas(df,somehow_useful_columns_but_droppable_anyway)
   #Quitamos las columnas con su mayoría de variables a nulos
	df = elimina_columnas(df,not_useful_columns_2)
  #Quitamos aquellas variables que no aprotan valor a la solución del problema
	df = elimina_columnas(df,columns_with_too_many_missing_values)
  #Price entre 20 y 120 €
	df = df [(df['Price']>20) & (df['Price']<150)]
  #Nos quedamos solo con los apartamentos
	df  = df[df['Property Type']=="Apartment"]
	df = elimina_columnas(df,['Property Type'])
	#Nos quedamos con aquellos pisos que se encuentran en Madrid
	df = df[df['City'].str.contains("Mad",na=False, case=False)]
  #Room type diferente a shared
	df = df [df['Room Type'] !="Shared room"]
  #Bed type igual a Real bed
	df = df [df['Bed Type'] =="Real Bed"]
	df = elimina_columnas(df,['Bed Type'])
	#Quitamos entradas con 0 baños
	df = df[df['Bathrooms'] != 0]
  #Aplicamos distancia harvesine para saber a qué distancia del centro están
	df = df .assign(centro =  lambda x: haversine_distance(x['Latitude'],x['Longitude'],centro_madrid_lat,centro_madrid_lon))
  #Eliminamos columnas las cuales hemos generado nuevas variables relacionado con la localización
	df = elimina_columnas(df,['City','Longitude','Latitude',"Neighbourhood", "Neighbourhood Group Cleansed", "Neighbourhood Cleansed"])


	#medias reviews
	reviews = df.loc[: , "Review Scores Rating":"Review Scores Value"]
	df['Average Review'] = reviews.mean(axis=1)

	df.drop(['Availability 30', 'Availability 60', 'Availability 90', 'Review Scores Rating',
				   'Review Scores Accuracy', 'Review Scores Cleanliness', 'Review Scores Checkin',
					 'Review Scores Communication', 'Review Scores Location', 'Review Scores Value',
					 'Accommodates'], axis=1, inplace=True)


	#Features , Amenities , Host verifications conteo
	df['Features'].fillna("", inplace=True)
	df['Amenities'].fillna("", inplace=True)
	df['Host Verifications'].fillna("", inplace=True)

	df['Features'] = df['Features'].apply(lambda x: len(str(x).split(',')))
	df['Amenities'] = df['Amenities'].apply(lambda x: len(str(x).split(',')))
	df['Host Verifications'] = df['Host Verifications'].apply(lambda x: len(str(x).split(',')))


	#años de inquilino
	from datetime import datetime
	df.dropna(subset=['Host Since'], inplace=True)
	df['Host Since'] = df['Host Since'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))
	df['Years Being Host'] = df['Host Since'].apply(lambda x: 2017 - x.year)
	df.drop(['Host Since'], axis=1, inplace=True)


	df["Host Response Time"].fillna(df["Host Response Time"].mode()[0], inplace=True)
	df.drop(['Experiences Offered'], axis=1, inplace=True)
	df.drop(['Country Code'], axis=1, inplace=True)


	# Variable categóricas a numéricas
	df['Host Response Time']  = df['Host Response Time'].astype(dummies_type_Host_Response_Time)
	df['Room Type'] 				  = df['Room Type'].astype(dummies_type_Room_Type)
	df['Cancellation Policy'] = df['Cancellation Policy'].astype(dummies_type_Cancellation_Policy)
	Host_Response_Time  = pd.get_dummies(df[['Host Response Time']], prefix='Host_Response_Time')
	Room_Type           = pd.get_dummies(df[['Room Type']], prefix='Room_Type')
	Cancellation_Policy = pd.get_dummies(df[['Cancellation Policy']], prefix='Cancellation_Policy')

	categorical_data = pd.concat([Host_Response_Time,Room_Type,Cancellation_Policy], axis=1, join='inner')
	df = pd.concat([df,categorical_data], axis=1)

	df = elimina_columnas(df,['Host Response Time','Room Type','Cancellation Policy'])



	# Rellenamos valores nulos sobrantes con la media
	df.fillna(df.mean(), inplace=True)


	#Nos quedamos con las imagenes de los registros válidos
	indices = df.index
	imagenes = imagenes[indices]

	return df , imagenes


In [ ]:
# Montamos (al ejecutarse en distintos dias el Drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Cargamos TRain
data_train   = pd.read_csv("/content/drive/MyDrive/train_1D.csv", sep=';')
images_train = np.load("/content/drive/MyDrive/train_images.npy")

#Cargamos Valid
data_valid  = pd.read_csv("/content/drive/MyDrive/valid_1D.csv", sep=';', decimal='.')
images_valid = np.load("/content/drive/MyDrive/valid_images.npy")


#Cargamos TEst
data_test   = pd.read_csv('/content/drive/MyDrive/test_1D.csv', sep=';', decimal='.')
images_test = np.load("/content/drive/MyDrive/test_images.npy")

In [ ]:
df_train_transform, images_train = transform(data_train,images_train)
df_valid_transform, images_valid = transform(data_valid,images_valid)
df_test_transform, images_test   = transform(data_test,images_test)

In [ ]:
print(f"Dimensiones del dataset de train antes del preprocesamiento: {data_train.shape}")
print(f"Dimensiones del dataset de valid antes del preprocesamiento: {data_valid.shape}")
print(f"Dimensiones del dataset de test antes del preprocesamiento: {data_test.shape}")


print(f"Dimensiones del dataset de train después del preprocesamiento: {df_train_transform.shape}")
print(f"Dimensiones del dataset de valid después del preprocesamiento: {df_valid_transform.shape}")
print(f"Dimensiones del dataset de test después del preprocesamiento: {df_test_transform.shape}")

In [ ]:
#Comprobamos que no haya valores nulos

df_train_transform.isna().sum()

In [ ]:
#Comprobamos que no haya valores nulos

df_train_transform.isnull().any()

Escalado

In [ ]:
#Funcion para mostrar tipo de columnas
def tipo_columnas(df):
    """Información sobre columnas numéricas y no numéricas de un dataframe.

	Parámetros:
	df -- dataframe

	Returns:
	num --  array con columnas numéricas('int','float')
	not_num -- array con columnas no numéricas.
	"""
    columnas_numericas = df.select_dtypes(include=['int', 'float','uint8']).columns
    print(f"Columnas numéricas: {len(columnas_numericas)} columnas.")
    print(', '.join(columnas_numericas))
    # Obtener las columnas de tipo no numérico
    columnas_no_numericas = df.select_dtypes(exclude=['int', 'float','uint8']).columns
    print(f"Columnas no númericas: {len(columnas_no_numericas)} columnas.")
    print(', '.join(columnas_no_numericas))
    return columnas_numericas , columnas_no_numericas

In [ ]:
#comprobamos numero de columnas numericas y categoricas

col_num , col_not_num = tipo_columnas(df_train_transform)

In [ ]:
# Separacion x e y
features = df_train_transform[col_num].columns.drop(['Price'])
num_features = len(features)
x_train  = df_train_transform[features].values
y_train  = df_train_transform['Price'].values
x_valid  = df_valid_transform[features].values
y_valid  = df_valid_transform['Price'].values
x_test   = df_test_transform[features].values
y_test   = df_test_transform['Price'].values


#Escalamos entre 0 y 1 (con los datos de train)
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

print(f'{x_train.shape} Dimensiones de train')
print(f'{x_valid.shape}  Dimensiones de valid')
print(f'{x_test.shape}  Dimensiones de test')
print(f'{y_train.shape} Dimensiones variable objetivo train')
print(f'{y_valid.shape} Dimensiones variable objetivo valid')
print(f'{y_test.shape}  Dimensiones variable objetivo test')



In [ ]:

#Creamos las Clases
y_train_multiclass = np.where(y_train < 45, 0 , (np.where(   np.logical_and(y_train >= 45,y_train <= 70) , 1 ,2)))
y_valid_multiclass = np.where(y_valid < 45, 0 , (np.where(   np.logical_and(y_valid >= 45,y_valid <= 70) , 1 ,2)))
y_test_multiclass  = np.where(y_test  < 45, 0  ,(np.where(   np.logical_and(y_test  >= 45,y_test  <= 70) , 1 ,2)))

In [ ]:
np.unique(y_train_multiclass, return_counts=True),np.unique(y_valid_multiclass, return_counts=True),np.unique(y_test_multiclass, return_counts=True)

In [ ]:
y_train_multiclass_cat =  to_categorical(y_train_multiclass)
y_valid_multiclass_cat =  to_categorical(y_valid_multiclass)
y_test_multiclass_cat  =  to_categorical(y_test_multiclass)

## 6. Modelo Basado en Datos

### 6.1 Funciones Auxiliares


Funciones de ayuda

Desnormalizar los resultados
Dibujar graficas
Resetar los pesos de modelo

In [ ]:
#Representación gráfica de la curvas loss / acurracy en train y validation
def plot_loss_curves(history):
  # """
  # Returns separate loss curves for training and validation metrics.

  # Args:
  #   history: TensorFlow model History object
  # """
  # loss = history.history['loss']
  # val_loss = history.history['val_loss']

  # accuracy = history.history['accuracy']
  # val_accuracy = history.history['val_accuracy']

  # epochs = range(len(history.history['loss']))

  # # Plot loss
  # plt.plot(epochs, loss, label='training_loss')
  # plt.plot(epochs, val_loss, label='val_loss')
  # plt.title('Loss')
  # plt.xlabel('Epochs')
  # plt.legend()

  # # Plot accuracy
  # plt.figure()
  # plt.plot(epochs, accuracy, label='training_accuracy')
  # plt.plot(epochs, val_accuracy, label='val_accuracy')
  # plt.title('Accuracy')
  # plt.xlabel('Epochs')
  # plt.legend();

  """
  Returns separate loss curves for training and validation metrics in two columns.

  Args:
    history: TensorFlow model History object
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Create subplots with 1 row and 2 columns
  plt.figure(figsize=(12, 5))  # Adjust the figure size as needed
  plt.subplots_adjust(wspace=0.3)  # Adjust the horizontal space between subplots

  # Plot loss in the first subplot (1, 2, 1)
  plt.subplot(1, 2, 1)
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy in the second subplot (1, 2, 2)
  plt.subplot(1, 2, 2)
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend()

  plt.show()

### 6.2 Modelo basado en Datos

In [ ]:
# Creamos un modelo para hacer primero clasificación multiclase
print(f"Input size: {x_train.shape[1]}")

num_clases =3
classificator1D = tf.keras.Sequential(name="Clasificador_datos_tabulares")
classificator1D.add(Dense(64, input_shape=(x_train.shape[1],), activation='relu'))
classificator1D.add(Dense(32, activation='relu'))
classificator1D.add(Dropout(0.2))
classificator1D.add(Dense(8, activation='relu'))
classificator1D.add(Dense(num_clases, activation='softmax')) # Sin función de activación

In [ ]:
classificator1D.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
classificator1D.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(classificator1D, show_shapes=True, show_layer_names=True)

In [ ]:
# Entrenamos un número fijo de épocas
epochs = 20
batch_size = 32

history_classificator1D = classificator1D.fit(x=x_train,
                        y=y_train_multiclass_cat,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_valid, y_valid_multiclass_cat),
                        verbose=1)

In [ ]:
classificator1D.evaluate(x_test, y_test_multiclass_cat, verbose=1)

In [ ]:

#Representación de modelos
from tensorflow.keras.utils import plot_model

In [ ]:
plot_loss_curves(history_classificator1D)

## 7. Modelo basado en imagenes

Red Neuronal Convolucional (CNN):
Una CNN es un tipo de arquitectura de red neuronal especialmente diseñada para procesar datos de imágenes.
Utiliza capas convolucionales para extraer características relevantes de las imágenes y capas de agrupación (pooling) para reducir la dimensionalidad.
Las capas convolucionales aplican filtros a pequeñas regiones de la imagen para detectar patrones como bordes, texturas y formas.
Las capas de agrupación reducen el tamaño de la representación de la imagen, manteniendo las características más importantes.
Las CNN son ampliamente utilizadas en tareas de clasificación de imágenes, detección de objetos y segmentación semántica.
Enfoque Funcional:
En lugar de construir una CNN de manera secuencial, podemos usar el enfoque funcional en Keras.
El enfoque funcional nos permite crear modelos más complejos con múltiples entradas o salidas.
Definimos las capas como funciones y luego las conectamos entre sí.
Creación del Modelo:
Crearemos una CNN con tres capas convolucionales: 16 filtros en la primera capa, 32 en la segunda y 64 en la tercera.
La primera capa debe recibir la dimensión de los datos de entrada (por ejemplo, el tamaño de la imagen).
Cambiaremos la última capa para adaptarla a nuestro problema específico (por ejemplo, clasificación de imágenes).

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input

# Tamaño de las imágenes
inputShape = (224, 224, 3)
chanDim = -1
# Filtros que vamos a aplicar en las capas convolucionales
filters=(16, 32, 64)
inputs = Input(shape=inputShape)
# Recorremos el bucle creando las diferentes capas
for (i, f) in enumerate(filters):
		# si es la primera capa indicamos el inputshape
  if i == 0:
	  x = inputs
	# CONV => BN => RELU => POOL
  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)

# Aplanamos los datos y ponemos una capa densa de 16 neuronas con activación relu
x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
# Añadimos otra capa densa de 4 neuronas y activación relu
x = Dense(4)(x)
x = Activation("relu")(x)
# Añadimos la última capa con 3 neuronas y función softmax para la calsificación
x = Dense(3, activation="softmax")(x)
# Construimos la red neuronal
model_images = Model(inputs, x)

In [ ]:
from keras.callbacks import LearningRateScheduler

def lr_time_based_decay(epoch, lr):
    decay = 1e-6  # Tasa de decaimiento
    return lr * 1 / (1 + decay * epoch)

history_model_images = model_images.fit(x=images_train, y=y_train_multiclass_cat,
                 validation_data=(images_valid, y_valid_multiclass_cat),
                 epochs=10, batch_size=32,
                 callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1)])

In [ ]:
plot_model(model_images, show_shapes=True, show_layer_names=True)

In [ ]:
plot_loss_curves(history_model_images)

## 8. Modelo Mixto

Late Fusion

In [ ]:
late_1D_train = classificator1D.predict(x_train)
late_1D_valid = classificator1D.predict(x_valid)
late_1D_test  = classificator1D.predict(x_test)

In [ ]:
print(late_1D_train.shape)
print(late_1D_valid.shape)
print(late_1D_test.shape)

In [ ]:
late_images_train = model_images.predict(images_train)
late_images_valid = model_images.predict(images_valid)
late_images_test  = model_images.predict(images_test)

In [ ]:
print(late_images_train.shape)
print(late_images_valid.shape)
print(late_images_test.shape)

In [ ]:
late_train_features = np.concatenate((late_1D_train, late_images_train), axis=1)
late_valid_features = np.concatenate((late_1D_valid, late_images_valid)  , axis=1)
late_test_features  = np.concatenate((late_1D_test,  late_images_test) , axis=1)

y_labels_train = np.concatenate((y_train_multiclass, y_valid_multiclass))
y_labels_test = y_test_multiclass

In [ ]:

#Combinación tipos de datos
from sklearn.svm import SVC
late_fuser = SVC(random_state=123)
late_fuser.fit(X=np.concatenate((late_train_features, late_valid_features)),y=y_labels_train)
acc_late = late_fuser.score(X=late_test_features, y=y_labels_test)
print(f"Late fusion SVC {round(acc_late*100,2)} % accuracy")